<a href="https://colab.research.google.com/github/OziomaEunice/Sentiment_GPT/blob/develop/Na%C3%AFveBayes_SentimentAnalysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Naïve Bayes for Sentiment Analysis**

In [ ]:
# install the necessary libraries
! pip install numpy
! pip install pandas
! pip install scikit-learn
! pip install nltk
! pip install openpyxl # for reading excel files

In [ ]:
# import the necessary libraries
import numpy as np
import pandas as pd
import nltk
import re
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib import style
style.use('ggplot')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
# Download the stopwords dataset
nltk.download('stopwords')

# Download wordnet dataset
nltk.download('wordnet')

# Download punkt dataset
nltk.download('punkt')

# Get the set of English stop words
stop_words = set(stopwords.words('english'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# since dataset is imported to my Google Colab (which will remain in here temporary)
# read excel file
df = pd.read_excel('/content/Tweets.xlsx') # for Twitter dataset
df2 = pd.read_excel('/content/IMDB_Dataset.xlsx') # for IMDb Movie Review dataset

In [ ]:
# check the number of rows and columns
df.shape

In [ ]:
df2.shape

In [ ]:
# show Twitter dataframe
df.head(10)

In [ ]:
# show IMDb dataset
df2.head(10)

In [ ]:
# datatype info
df.info()

print('\n***========================***\n')

df2.info()

In [ ]:
# counting the number of missing values in the dataset
df.isnull().sum()

In [ ]:
df2.isnull().sum()

### **Visualisation (before preprocessing)**

#### **Twitter**

In [ ]:
# Visualize sentiment distribution before preprocessing
plt.figure(figsize=(10, 6))
sns.countplot(x='airline_sentiment', data=df, palette=['#99FF99', '#3399FF', '#FF6666'])
plt.title('Sentiment Distribution Twitter US Airline Dataset\n (Before Preprocessing)')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualise sentiment in wordcloud
def generate_wordcloud(text, title):
    wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stop_words,
                min_font_size = 10).generate(text)

    # Plot the WordCloud image
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.title(title)
    plt.show()

# Visualize word cloud before preprocessing for Twitter dataset
generate_wordcloud(' '.join(df['text'].values), 'Twitter - Word Cloud (Before Preprocessing)')

#### **IMDb**

In [ ]:
# Visualize sentiment distribution before preprocessing
plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment', data=df2, palette=['#3399FF', '#FF6666'])
plt.title('Sentiment Distribution IMDb Movie Review Dataset\n (Before Preprocessing)')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualize word cloud before preprocessing for IMDb dataset
generate_wordcloud(' '.join(df2['review'].values), 'IMDb - Word Cloud (Before Preprocessing)')

## **Preprocessing Dataset**

### **Handling Missing Values**

In [ ]:
# drop columns that are not needed for processing data
# In this case, for the Twitter dataset
df = df.drop(columns=['tweet_id', 'airline_sentiment_gold', 'negativereason', 'negativereason_confidence', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location', 'user_timezone'])

In [ ]:
df.isnull().sum()

In [ ]:
df.head(10)

### **Cleaning Dataset**

In [ ]:
# clean (preprocess) the Twitter dataset
def cleanData1(text, min_word_length = 3):
  text = text.lower()
  text = re.sub(r'@[A-Za-z0-9]+', "", text) # this informs Python the the mentions in text must be substituted with an empty string
  text = re.sub(r'#', "", text) # removing #
  text = re.sub(r'RT[\s]+', "", text) # removing retweets
  text = re.sub(r'https?:\/\/\S+', "", text) # removing links
  text = ' '.join(word for word in text.split() if len(word) >= min_word_length and word not in stop_words) # Remove short words

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
  for p in punctuations:
      text = text.replace(p,'') #Removing punctuations

  # Lemmatize the words
  words = nltk.word_tokenize(text)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  text = ' '.join(lemmatized_words)

  return text

In [ ]:
# clean (preprocess) the IMDb Movie Review dataset
def cleanData2(review, min_word_length = 3):
  review = review.lower()
  review = re.sub(r'@[A-Za-z0-9]+', "", review) # this informs Python the the mentions in text must be substituted with an empty string
  review = re.sub(r'#', "", review) # removing #
  review = re.sub(r'RT[\s]+', "", review) # removing retweets
  review = re.sub(r'https?:\/\/\S+', "", review) # removing links
  review = ' '.join(word for word in review.split() if len(word) >= min_word_length and word not in stop_words) # Remove short words

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
  for p in punctuations:
      review = review.replace(p,'') #Removing punctuations

  # Lemmatize the words
  words = nltk.word_tokenize(review)
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  review= ' '.join(lemmatized_words)

  return review

In [ ]:
# df['text'] = df['text'].apply(cleanData)

# Apply the cleanData function to all text columns in the dataframe
df = df.applymap(lambda x: cleanData1(x) if isinstance(x, str) else x)

In [ ]:
df2 = df2.applymap(lambda x: cleanData2(x) if isinstance(x, str) else x)

### **Visualisation (after preprocessing)**

#### **Twitter**

In [ ]:
# Visualize sentiment distribution before preprocessing
plt.figure(figsize=(10, 6))
sns.countplot(x='airline_sentiment', data=df, palette=['#99FF99', '#3399FF', '#FF6666'])
plt.title('Sentiment Distribution Twitter US Airline Dataset\n (After Preprocessing)')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualize word cloud after preprocessing for Twitter dataset
generate_wordcloud(' '.join(df['text'].values), 'Twitter - Word Cloud (After Preprocessing)')

#### **IMDb**

In [ ]:
# Visualize sentiment distribution after preprocessing
plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment', data=df2, palette=['#3399FF', '#FF6666'])
plt.title('Sentiment Distribution IMDb Movie Review Dataset\n (After Preprocessing)')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

In [ ]:
# Visualize word cloud after preprocessing for IMDb dataset
generate_wordcloud(' '.join(df2['review'].values), 'IMDb - Word Cloud (After Preprocessing)')

### **Converting labels into integers or numeric values for ML**
**The purpose of this conversion is so that machines can understand the labels, since they aren't able to understand text data**

In [ ]:
# create a new column called data and link the airline_sentiment to it
# (this 'label' column will have numbers indicating 1, 0, -1
# (positive, neutral, negative)
df['label'] = df['airline_sentiment'].apply(lambda x: int(1) if str(x) == 'positive' else (0 if x == 'neutral' else -1))
df2['label'] = df2['sentiment'].apply(lambda x: int(1) if x == 'positive' else (0 if x == 'neutral' else -1))

In [ ]:
df[['airline_sentiment', 'text', 'label']].head(20)

## **Feature Extraction: TF-IDF Vectorisation**

### **Split the Dataset into Training and Testing**

In [ ]:
text_train, text_test, label_train, label_test = train_test_split(
                                          df['text'].values,
                                          df['label'].values,
                                          test_size = 0.2,
                                          random_state = 42,
                                          stratify=df['label'].values
                                      )

In [ ]:
print(len(text_train), len(text_test), len(label_train) , len(label_test))

### **TF-IDF**

In [ ]:
# Initialize the TF-IDF vectorizer and store it in a variable
tfidf_vector = TfidfVectorizer()

# Transform the training and testing sets
tfidf_train = tfidf_vector.fit_transform(text_train)
tfidf_test = tfidf_vector.transform(text_test)

## **Training**

In [ ]:
# train the Naïve Bayes (Multinomial) model
naive_bayes = MultinomialNB()

naive_bayes.fit(tfidf_train, label_train)

## **Testing**

In [ ]:
# test the Naïve Bayes (Multinomial) model
tfidf_test_imdb = tfidf_vector.transform(df2['review'].values)

predictions_imdb = naive_bayes.predict(tfidf_test_imdb)

print('==============================================')
print('\nClassification Report\n')
print('==============================================')

accuracy = accuracy_score(df2['label'].values, predictions_imdb)
accuracy_perc = accuracy * 100

print(f"Accuracy on IMDb dataset: {accuracy_perc:.2f}%")
print('---------------------------\n')

# print(classification_report(label_test,predictions))
print(classification_report(df2['label'], predictions_imdb, zero_division=1))

In [ ]:
df2.head(20)

### **Confusion Matrix**

In [ ]:
print(confusion_matrix(df2['label'], predictions_imdb))

In [ ]:
style.use('classic')
cm = confusion_matrix(df2['label'], predictions_imdb)

# Display confusion matrix
display_cm = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['positive', 'neutral', 'negative'])
display_cm.plot(cmap='viridis', values_format='d')

plt.title('Confusion Matrix')
plt.show()

## **Error Analysis**

In [ ]:
# create an error analysis dataframe to
# compare the model's predictions with the true labels
error_analysis_df = pd.DataFrame({
    'Review': df2['review'].values,
    'True Label': df2['label'].values,
    'Predicted Label': predictions_imdb
})

In [ ]:
# identify misclassification
misclassifications = error_analysis_df[error_analysis_df['True Label'] != error_analysis_df['Predicted Label']]

In [ ]:
# display the first 10 misclassifications
for index, row in misclassifications.head(10).iterrows():
    print(f"Review: {row['Review']}")
    print(f"True Label: {row['True Label']}, Predicted Label: {row['Predicted Label']}")
    print('---')

## **Hyperparameter**

### **Using GridSearchCV**

In [ ]:
# define the hyperparameter grid
parameter_grid ={
    'alpha': [0.01, 0.1, 1, 10],
    'fit_prior': [True, False]
}

In [ ]:
grid = GridSearchCV(naive_bayes, parameter_grid, cv=5)
grid.fit(tfidf_train, label_train)

In [ ]:
# Print the best hyperparameters
print("Best Hyperparameters:", grid.best_params_)

# print how the model looks after hyper-parameter tuning
print("Best Estimator:", grid.best_estimator_)

In [ ]:
# Get the best Naïve Bayes model
best_nb_model = grid.best_estimator_

# Test the best SVM model
tfidf_test_imdb = tfidf_vector.transform(df2['review'].values)
predictions_imdb_best = best_nb_model.predict(tfidf_test_imdb)

In [ ]:
# Display the classification report and confusion matrix for the best model
print('\n==============================================')
print('Classification Report for Best Model\n')
print('==============================================')

accuracy = accuracy_score(df2['label'].values, predictions_imdb_best)
accuracy_perc = accuracy * 100

print(f"Accuracy on IMDb dataset: {accuracy_perc:.2f}%")
print('---------------------------\n')

print(classification_report(df2['label'], predictions_imdb_best, zero_division=1))

# Calculate confusion matrix for best model
cm_best = confusion_matrix(df2['label'], predictions_imdb_best)

# Display confusion matrix for best model
display_cm_best = ConfusionMatrixDisplay(confusion_matrix=cm_best, display_labels=['positive', 'neutral', 'negative'])
display_cm_best.plot(cmap='viridis', values_format='d')

plt.title('Confusion Matrix for Best Model')
plt.show()